In [ ]:
# Data Preprocessing


import numpy as np 
import pandas as pd 
import csv


def addrow(full_band_dict, object_id, mjd, wfile, label):
	new_row = [object_id, mjd]
	for band in ['u','g','r','i','z','y']:
		if band not in full_band_dict:
			new_row += [None, None]
		else:
			new_row += full_band_dict[band]
	new_row.append(label)
	wfile.writerow(new_row)

data = pd.read_csv('../input/divided-plasticc-testset/plasticc_test_set_batch2_2.csv') #csv file need to be converted
meta = pd.read_csv('../input/plasticcunblindeddatasets/plasticc_test_metadata.csv') # metadata

meta = meta[(meta['ddf_bool']<1)]
WDF_obj = meta.groupby('object_id')['object_id'].apply(int)
# WDF_label = meta.groupby('object_id')['target'].apply(list)

print(WDF_obj)

file = open('test_set_converted_batch2_2.csv','w')

wfile = csv.writer(file)
wfile.writerow(['id','mjd','u','u_err','g','g_err','r','r_err','i','i_err','z','z_err','y','y_err','label'])
band_dict = {0:'u',1:'g',2:'r',3:'i',4:'z',5:'y'}
last_id = None
last_mjd = None
last_label = None
full_band_dict = {}
n = 0
check = 0
while n < len(data):
	
	if data['object_id'][n] in WDF_obj:
		if data['object_id'][n]==last_id:
			delta_mjd = data['mjd'][n] - last_mjd
			if delta_mjd >=0.5:
				addrow(full_band_dict, last_id, last_mjd, wfile, last_label)
				full_band_dict = {}
				last_mjd = data['mjd'][n]
			if band_dict[data['passband'][n]] not in full_band_dict:
				full_band_dict[band_dict[data['passband'][n]]] = [data['flux'][n],data['flux_err'][n]]
		else:
			if last_id != None and last_mjd != None:
				addrow(full_band_dict, last_id, last_mjd, wfile, last_label)
			last_id = data['object_id'][n]
			last_mjd = data['mjd'][n]
			label = int(meta[meta.object_id == last_id]['target'])
			if label == 88:
				last_label = 1
			else:
				last_label = 0
			print(last_id)
			check +=1
			n -=1
	n +=1

print('check:', check)
print('test num:', len(WDF_obj))

In [ ]:
# divide origninal files

import pandas as pd 
import numpy as np 


input_path = '../input/plasticcunblindeddatasets/'
file = 'plasticc_test_set_batch2'
file_path = input_path+file
# data = pd.read_csv(file_path+'.csv')


id_set = set()
n = 0
for chunk in pd.read_csv(file_path+'.csv', chunksize = 2000000):
    print(n)
    chunk_id_list = []
    groups = chunk.groupby(chunk['object_id'])
    num_group = groups.ngroups
    for _id, group in groups:
        chunk_id_list.append(_id)
    if chunk_id_list[0] in id_set:
        add_part = chunk[chunk.object_id == chunk_id_list[0]]
        add_part.to_csv(file+'_'+str(n-1)+'.csv', mode='a', header=False)
        chunk = chunk[~(chunk.object_id == chunk_id_list[0])]
    chunk.to_csv(file+'_'+str(n)+'.csv')
    id_set.update(chunk_id_list)  
    n +=1

In [ ]:
# test convert dataframe using  pivot
import pandas as pd 
import numpy as np 

df  = pd.read_csv('../input/PLAsTiCC-2018/test_set_batch2.csv')
df[:10]

In [ ]:
df = df.pivot_table(index = ['object_id','mjd'], columns = ['passband'],values = ['flux','flux_err']).reset_index()
df[:10]

In [ ]:
df.columns = ['object_id','mjd','u_flux','g_flux','r_flux','i_flux','z_flux','y_flux','u_flux_err','g_flux_err','r_flux_err','i_flux_err','z_flux_err','y_flux_err']
df[:10]

In [ ]:
# Use .rename first, then use map() to combine multiIndex header
# df = df.rename(columns = {0:'u',1:'g',2:'r',3:'i',4:'z',5:'y'})
# df.columns = list(map('_'.join, df3.columns))
df.to_csv('converted_test_batch2.csv')


In [ ]:
# df1 = pd.pivot(df, columns="passband", values=["flux","flux_err"])
# df['flux,flux_err'] = df['flux'].map(str)+','+df['flux_err'].map(str)
# df.drop(labels = ['flux','flux_err'], axis = 1)
# # df[:10]
# df1 = df.set_index(['object_id','mjd','detected','passband'])['flux,flux_err'].unstack().rename_axis(columns=None).reset_index()

# df1[:10]